# pyFMでmovie lensしてみる

In [1]:
from movielens_loader import MovielensLoader
from pathlib import Path
loader=MovielensLoader(data_dir=Path("./ml-100k"))

In [2]:
X_train,y_train,X_test,y_test=loader.create_dataset(include_context_features=True)

In [3]:
X_train.toarray().shape,X_test.toarray().shape

((90570, 2646), (9430, 2646))

In [4]:
from pyfm import pylibfm
fm = pylibfm.FM(num_factors=10, num_iter=100, verbose=True, task="regression", initial_learning_rate=0.001, learning_rate_schedule="optimal")

In [5]:
fm.fit(X_train,y_train)

Creating validation dataset of 0.01 of training for adaptive regularization
-- Epoch 1
Training MSE: 0.57682
-- Epoch 2
Training MSE: 0.49032
-- Epoch 3
Training MSE: 0.45951
-- Epoch 4
Training MSE: 0.44543
-- Epoch 5
Training MSE: 0.43773
-- Epoch 6
Training MSE: 0.43291
-- Epoch 7
Training MSE: 0.42967
-- Epoch 8
Training MSE: 0.42678
-- Epoch 9
Training MSE: 0.42442
-- Epoch 10
Training MSE: 0.42235
-- Epoch 11
Training MSE: 0.42047
-- Epoch 12
Training MSE: 0.41886
-- Epoch 13
Training MSE: 0.41722
-- Epoch 14
Training MSE: 0.41531
-- Epoch 15
Training MSE: 0.41420
-- Epoch 16
Training MSE: 0.41230
-- Epoch 17
Training MSE: 0.41080
-- Epoch 18
Training MSE: 0.40934
-- Epoch 19
Training MSE: 0.40803
-- Epoch 20
Training MSE: 0.40633
-- Epoch 21
Training MSE: 0.40486
-- Epoch 22
Training MSE: 0.40335
-- Epoch 23
Training MSE: 0.40191
-- Epoch 24
Training MSE: 0.40036
-- Epoch 25
Training MSE: 0.39874
-- Epoch 26
Training MSE: 0.39725
-- Epoch 27
Training MSE: 0.39545
-- Epoch 28
Tra

In [14]:
from sklearn.metrics import mean_squared_error
import numpy as np
preds = fm.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test,preds))
print(f"FM RMSE: {rmse:.4f}")

FM RMSE: 0.9427


In [7]:
# baseline
y_train_mean=y_train.mean()*np.ones(len(y_test))
print(f"[Baseline] : RMSE = {np.sqrt(mean_squared_error(y_train_mean, y_test)):0.4f}")

[Baseline] : RMSE = 1.1220


In [19]:
import lightgbm as lgb
gbm = lgb.LGBMRegressor()
gbm.fit(X_train.toarray(), y_train)
preds= gbm.predict(X_test.toarray())
rmse = np.sqrt(mean_squared_error(y_test,preds))
print(f"LightGBM RMSE: {rmse:.4f}")

LightGBM RMSE: 1.0315
